## Рекомендательные системы

#### Курсовая работа

In [4]:
import pandas as pd
import numpy as np

import lightgbm as lgb
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from data.metrics import precision_at_k, recall_at_k
from data.utils import prefilter_items
from data.recommenders import MainRecommender

import warnings
warnings.filterwarnings('ignore')


##### Загрузка данных

In [5]:
data = pd.read_csv('data/retail_train.csv')
data.head(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.3,1631,1,0.0,0.0


In [6]:
test = pd.read_csv('data/retail_test1.csv')
test.head(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0
2,2070,41652857291,664,995242,5,9.10,311,-0.6,46,96,0.0,0.0


In [8]:
item_features = pd.read_csv('data/product.csv')
user_features = pd.read_csv('data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

item_features.head(3)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,


In [9]:
user_features.head(3)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8



##### Схема обучения и валидации:
-- давние покупки -- | -- 6 недель -- | -- 3 недель --


In [10]:
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

In [11]:
users_lvl_1 = data_train_lvl_1.user_id.unique()
users_lvl_2 = data_val_lvl_1.user_id.unique()
users_lvl_3 = data_val_lvl_2.user_id.unique()

new_users_lvl_2 = list(set(users_lvl_2) - set(users_lvl_1))
new_users_lvl_3 = list(set(users_lvl_3) - (set(users_lvl_1) | set(users_lvl_2)))

add_to_lvl_2 = list(set(users_lvl_3) - (set(users_lvl_2)))

new_users_lvl_2, new_users_lvl_3, len(add_to_lvl_2)

([1984], [], 126)

In [12]:
# Уменьшение размерности
n_items_before = data_train_lvl_1['item_id'].nunique()
data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)
n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


##### Генерация и отбор новых признаков для модели второго уровня

In [13]:
user_features['age_desc'].replace(
    {'19-24': 22, '25-34': 30, '35-44': 40, '45-54': 50, '55-64': 60, '65+': 70},
    inplace=True)

user_features['marital_status_code'].replace(
    {'U': 0, 'A': 1, 'B': 2}, inplace=True)

user_features['income_desc'].replace(
    {'Under 15K': 10, '15-24K': 20, '25-34K':30, '35-49K': 40,
     '50-74K': 62, '75-99K': 87, '100-124K': 112, '125-149K': 137, 
     '150-174K': 162, '175-199K': 187, '200-249K': 225, '250K+':275}, inplace=True)

user_features['homeowner_desc'].replace(
    {'Unknown': 0, 'Probable Renter': 1, 'Renter': 2,
     'Probable Owner': 3, 'Homeowner': 4}, inplace=True)

user_features['hh_comp_desc'].replace(
    {'Unknown': 0, 'Single Male': 1, 'Single Female': 2,
     '1 Adult Kids': 3, '2 Adults No Kids': 4, '2 Adults Kids':5},inplace=True)

user_features['household_size_desc'].replace({'5+': 5}, inplace=True) 

user_features['kid_category_desc'].replace(
    {'None/Unknown': 0, '3+': 3}, inplace=True)

user_features.head(3)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,70,1,40,4,4,2,0,1
1,50,1,62,4,4,2,0,7
2,30,0,30,0,5,3,1,8


In [14]:
names = ['manufacturer', 'department', 'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product']
for name in names:
    new_name = name + '_freq'
    a = item_features[name].value_counts()
    ind = a.index.tolist()
    for i in ind:
        item_features.loc[item_features[name] == i, new_name] = a[i]

item_features['brand'] = np.where(item_features['brand']=='Private', 0, 1)

commodities = item_features.commodity_desc.value_counts()
commodities_list = commodities.keys().tolist()
for i, name in enumerate(commodities_list):
    item_features.loc[item_features['commodity_desc'] == name, 'commodity_category'] = i

item_features.head(3)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,manufacturer_freq,department_freq,commodity_desc_freq,sub_commodity_desc_freq,curr_size_of_product_freq,commodity_category
0,25671,2,GROCERY,1,FRZN ICE,ICE - CRUSHED/CUBED,22 LB,1411.0,39021.0,29.0,29.0,12.0,270.0
1,26081,2,MISC. TRANS.,1,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,,1411.0,490.0,490.0,429.0,30607.0,56.0
2,26093,69,PASTRY,0,BREAD,BREAD:ITALIAN/FRENCH,,12676.0,2149.0,343.0,107.0,30607.0,91.0


In [15]:
def get_new_features(data_train_lvl_1, n_factors=20):
    
    # час совершения транзакции
    data = data_train_lvl_1.copy()
    data['hour'] = data['trans_time'] // 100
    user_item_features = data.groupby(['user_id', 'item_id'])['hour'].median().reset_index()
    user_item_features.columns = ['user_id', 'item_id', 'median_sales_hour']
    
    # день недели совершения транзакции
    data['weekday'] = data['day'] % 7
    df = data.groupby(['user_id', 'item_id'])['weekday'].median().reset_index()
    df.columns = ['user_id', 'item_id', 'median_weekday']
    user_item_features = user_item_features.merge(df, on=['user_id', 'item_id'])
    
    # cреднее кол-во дней между покупками
    df = data.groupby('user_id')['day'].nunique().reset_index()
    df['mean_visits_interval'] = (data.groupby('user_id')['day'].max() - data.groupby('user_id')['day'].min()) / df['day']
    user_item_features = user_item_features.merge(df[['user_id', 'mean_visits_interval']], on=['user_id'])
    
    # средний чек корзины клиента
    df = data.groupby(['user_id', 'basket_id'])['sales_value'].sum().reset_index()
    df = df.groupby('user_id')['sales_value'].mean().reset_index()
    df.columns = ['user_id', 'mean_check']
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    # кол-во магазинов, в которых продавался товар
    df = data.groupby(['item_id'])['store_id'].nunique().reset_index()
    df.columns = ['item_id', 'n_stores']
    user_item_features = user_item_features.merge(df, on=['item_id'])
    
    # кол-во уникальных товаров, купленных клиентом
    df = data.groupby(['user_id'])['item_id'].nunique().reset_index()
    df.columns = ['user_id', 'n_items']
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    # кол-во транзакций клиента
    df = data.groupby(['user_id'])['item_id'].count().reset_index()
    df.columns = ['user_id', 'n_transactions']
    user_item_features = user_item_features.merge(df, on=['user_id'])
    
    # mean / max / std кол-ва уникальных товаров в корзине клиента
    df = data.groupby(['user_id', 'basket_id'])['item_id'].nunique().reset_index()
    df1 = df.groupby('user_id')['item_id'].mean().reset_index()
    df1.columns = ['user_id', 'mean_n_items_basket']
    user_item_features = user_item_features.merge(df1, on=['user_id'])

    df2 = df.groupby('user_id')['item_id'].max().reset_index()
    df2.columns = ['user_id', 'max_n_items_basket']
    user_item_features = user_item_features.merge(df2, on=['user_id'])

    df3 = df.groupby('user_id')['item_id'].std().reset_index()
    df3.columns = ['user_id', 'std_n_items_basket']
    user_item_features = user_item_features.merge(df3, on=['user_id'])
    
    # mean / max / std кол-ва уникальных категорий в корзине клиента
    data = data.merge(item_features[['item_id', 'commodity_desc']], on=['item_id'])
    df = data.groupby(['user_id', 'basket_id'])['commodity_desc'].nunique().reset_index()
    df1 = df.groupby('user_id')['commodity_desc'].mean().reset_index()
    df1.columns = ['user_id', 'mean_n_item_categories_basket']
    user_item_features = user_item_features.merge(df1, on=['user_id'])

    df2 = df.groupby('user_id')['commodity_desc'].max().reset_index()
    df2.columns = ['user_id', 'max_n_item_categories_basket']
    user_item_features = user_item_features.merge(df2, on=['user_id'])

    df3 = df.groupby('user_id')['commodity_desc'].std().reset_index()
    df3.columns = ['user_id', 'std_n_item_categories_basket']
    user_item_features = user_item_features.merge(df3, on=['user_id'])
    
    return user_item_features

In [16]:
user_item_features = get_new_features(data_train_lvl_1)
user_item_features.head(3)

,user_id,item_id,median_sales_hour,median_weekday,mean_visits_interval,mean_check,n_stores,n_items,n_transactions,mean_n_items_basket,max_n_items_basket,std_n_items_basket,mean_n_item_categories_basket,max_n_item_categories_basket,std_n_item_categories_basket
0,1,823721,13.0,4.0,NaN,37.4205,94,172,661,8.533333,18,4.928185,7.527273,16,3.915092
1,1,823990,15.0,6.0,NaN,37.4205,95,172,661,8.533333,18,4.928185,7.527273,16,3.915092
2,1,825123,17.0,3.0,NaN,37.4205,29,172,661,8.533333,18,4.928185,7.527273,16,3.915092


In [17]:
def get_candidates(data_train_lvl_1, data_train_lvl_2, N, add_to_lvl_2):
    recommender = MainRecommender(data_train_lvl_1)

    users_lvl_1 = data_train_lvl_1['user_id'].unique()
    users_lvl_2 = data_train_lvl_2['user_id'].unique().tolist()
    if add_to_lvl_2:
        users_lvl_2 += add_to_lvl_2

    current_users = list(set(users_lvl_2) & set(users_lvl_1))    
    new_users = list(set(users_lvl_2) - set(users_lvl_1))

    df = pd.DataFrame(users_lvl_2, columns=['user_id'])
    cond_1 = df['user_id'].isin(current_users)
    df.loc[cond_1, 'candidates'] = df.loc[cond_1, 'user_id'].apply(
        lambda x: recommender.get_own_recommendations(x, N))

    if new_users:
        cond_2 = df['user_id'].isin(new_users)
        df.loc[cond_2, 'candidates'] = df.loc[cond_2, 'user_id'].apply(
            lambda x: recommender.overall_top_purchases[:N])
        
    return df

In [18]:
def get_targets_lvl_2(data_train_lvl_1, data_train_lvl_2, user_item_features, N, add_to_lvl_2=None):
    
    users_lvl_2 = get_candidates(data_train_lvl_1, data_train_lvl_2, N, add_to_lvl_2)
    
    df = pd.DataFrame({'user_id': users_lvl_2['user_id'].values.repeat(N),
                       'item_id': np.concatenate(users_lvl_2['candidates'].values)})

    targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
    targets_lvl_2['target'] = 1 

    targets_lvl_2 = df.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')
    targets_lvl_2['target'].fillna(0, inplace= True)
    
    targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
    targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')
    targets_lvl_2 = targets_lvl_2.merge(
        user_item_features, on=['user_id', 'item_id'], how='left')
    
    return targets_lvl_2

In [19]:
N = 500
targets_lvl_2 = get_targets_lvl_2(data_train_lvl_1, data_train_lvl_2, user_item_features, N, add_to_lvl_2)

print(f'Число пользователей: {targets_lvl_2.user_id.nunique()}')
print(f'Среднее число покупок: {round(targets_lvl_2["target"].mean(), 4)}')

targets_lvl_2.head(3)

100%|██████████| 5001/5001 [00:00<00:00, 31785.62it/s]


Число пользователей: 2280
Среднее число покупок: 0.0388


,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,manufacturer_freq,...,mean_check,n_stores,n_items,n_transactions,mean_n_items_basket,max_n_items_basket,std_n_items_basket,mean_n_item_categories_basket,max_n_item_categories_basket,std_n_item_categories_basket
0,2070,1105426,0.0,69,DELI,0,SANDWICHES,SANDWICHES - (COLD),,12676.0,...,23.48875,27.0,272.0,592.0,3.710938,27.0,5.431249,3.428571,19.0,4.497422
1,2070,1097350,0.0,2468,GROCERY,1,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,299.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2070,879194,0.0,69,DRUG GM,0,DIAPERS & DISPOSABLES,BABY DIAPERS,14 CT,12676.0,...,23.48875,9.0,272.0,592.0,3.710938,27.0,5.431249,3.428571,19.0,4.497422


In [20]:
SELECTED_FEATURES_NAMES = targets_lvl_2.columns.tolist()
to_drop=['user_id', 'item_id', 'target', 'manufacturer', 'department']
categorical_features = ['marital_status_code', 'homeowner_desc',
                        'hh_comp_desc', 'manufacturer', 'commodity_category',
                        'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product',
                        'household_size_desc', 'kid_category_desc']


SELECTED_FEATURES_NAMES = [_ for _ in SELECTED_FEATURES_NAMES if not _ in (categorical_features+to_drop)]

#### Обучение моделей

##### CatBoost

In [22]:
X_train, X_valid, y_train, y_valid = train_test_split(targets_lvl_2[SELECTED_FEATURES_NAMES].fillna(0),
                                                          targets_lvl_2[['target']],
                                                          test_size=0.2, random_state=27,
                                                          stratify=targets_lvl_2[['target']])
dtrain = Pool(data=X_train, label=y_train)
dvalid = Pool(data=X_valid, label=y_valid)
params_cb = {"n_estimators":5000,
             "learning_rate": 0.1,
             "loss_function": "Logloss",
             "eval_metric": "AUC",
             "task_type": "GPU",
             "max_bin": 30,
             "early_stopping_rounds": 100,
             "verbose": 500,
             "max_depth": 10,
             "l2_leaf_reg": 80,
             "thread_count": 6,
             "random_seed": 21} 

model_cb = CatBoostClassifier(**params_cb)
model_cb.fit(dtrain,
             eval_set=[dvalid]
            )

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 0.8510888	test: 0.8470943	best: 0.8470943 (0)	total: 131ms	remaining: 10m 56s
500:	learn: 0.9190818	test: 0.8968893	best: 0.8968893 (500)	total: 1m 12s	remaining: 10m 47s
1000:	learn: 0.9347711	test: 0.9038863	best: 0.9038863 (1000)	total: 2m 56s	remaining: 11m 45s
1500:	learn: 0.9432467	test: 0.9071738	best: 0.9071738 (1500)	total: 4m 44s	remaining: 11m 3s
2000:	learn: 0.9487096	test: 0.9089215	best: 0.9089267 (1999)	total: 6m 13s	remaining: 9m 19s
2500:	learn: 0.9526384	test: 0.9097150	best: 0.9097305 (2445)	total: 8m	remaining: 7m 59s
bestTest = 0.9099838436
bestIteration = 2848
Shrink model to first 2849 iterations.


##### LightGBM

In [32]:
X_train, X_valid, y_train, y_valid = train_test_split(targets_lvl_2[SELECTED_FEATURES_NAMES].fillna(0),
                                                      targets_lvl_2[['target']],
                                                      test_size=0.2, random_state=27,
                                                      stratify=targets_lvl_2[['target']])

dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)

params_lgb = {"boosting_type": "gbrt",
              "objective": "binary",
              "metric": "auc",
              "num_boost_round": 10000,
              "learning_rate": 0.1,
              "max_depth": 10,
              "num_leaves":25,
              "n_estimators": 5000,
              "n_jobs": 6,
              "seed": 123
              } 

model_lgb = lgb.train(params=params_lgb,
                      train_set=dtrain,
                      valid_sets=[dtrain, dvalid],
                      verbose_eval=500,
                      early_stopping_rounds=100)

[LightGBM] [Info] Number of positive: 35723, number of negative: 885897
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.049959 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3056
[LightGBM] [Info] Number of data points in the train set: 921620, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.038761 -> initscore=-3.210806
[LightGBM] [Info] Start training from score -3.210806
Training until validation scores don't improve for 100 rounds
[500]	training's auc: 0.914951	valid_1's auc: 0.898214
[1000]	training's auc: 0.929243	valid_1's auc: 0.905046
[1500]	training's auc: 0.93792	valid_1's auc: 0.908699
[2000]	training's auc: 0.943532	valid_1's auc: 0.910799
[2500]	training's auc: 0.947749	valid_1's auc: 0.912443
[3000]	training's auc: 0.95108	valid_1's auc: 0.913283
[3500]	training's auc: 0.953866	valid_1's 

In [33]:
predictions_cb_train = model_cb.predict_proba(targets_lvl_2[SELECTED_FEATURES_NAMES].fillna(0))[:, 1]
predictions_lgb_train = model_lgb.predict(targets_lvl_2[SELECTED_FEATURES_NAMES].fillna(0))

In [34]:
preds_train_2_mods = 0.6*predictions_lgb_train + 0.4* predictions_cb_train
roc_auc_score(targets_lvl_2['target'], preds_train_2_mods)

0.9475028562820907

##### Прогнозирование и метрики

In [35]:
def get_predictions(targets_lvl_2, raw_predictions, prefix='lgb'): 
    df = targets_lvl_2[['user_id', 'item_id']]
    df['predictions'] = raw_predictions

    df = df.groupby(['user_id', 'item_id'])['predictions'].median().reset_index()
    df = df.sort_values(['predictions'], ascending=False).groupby(['user_id']).head(5)

    df = df.groupby('user_id')['item_id'].unique().reset_index()
    df.columns = ['user_id', prefix + '_recommendations']
    
    return df

In [36]:
def get_results(data_val_lvl_2, targets_lvl_2, preds_lgb, preds_cb, combined_preds):
    result = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
    result.columns=['user_id', 'actual']
    
    prefixes = ['lgb', 'cb', '2_mods']
    predictions = [preds_lgb, preds_cb, combined_preds]
    
    for i, preds in enumerate(predictions):
        df = get_predictions(targets_lvl_2, preds, prefixes[i])
        result = result.merge(df, on='user_id', how='left')

    return result

In [37]:


result_lvl_2 = get_results(data_val_lvl_2, targets_lvl_2, 
                           predictions_lgb_train,
                           predictions_cb_train, 
                           preds_train_2_mods)
result_lvl_2.head(3)

,user_id,actual,lgb_recommendations,cb_recommendations,2_mods_recommendations
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1082212, 9527558, 1082185, 865456, 8091601]","[8293439, 872137, 940947, 8091601, 1004906]","[1082212, 8293439, 9527558, 1082185, 940947]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1082185, 1106523, 866211, 1068719, 979707]","[1082185, 1106523, 990797, 883003, 1101010]","[1082185, 1106523, 990797, 1068719, 866211]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1024306, 1098844, 6548453, 1082185, 845208]","[1024306, 1098844, 6548453, 1082185, 900802]","[1024306, 1098844, 6548453, 1082185, 845208]"


In [38]:
result_lvl_2.apply(lambda row: precision_at_k(row['cb_recommendations'], row['actual'], 5), axis=1).mean()

0.28638589618021437

In [39]:
result_lvl_2.apply(lambda row: precision_at_k(row['lgb_recommendations'], row['actual'], 5), axis=1).mean()

0.27972575905974434

In [40]:
result_lvl_2.apply(lambda row: precision_at_k(row['2_mods_recommendations'], row['actual'], 5), axis=1).mean()

0.2886385896180207

In [41]:

validation_weeks = 6
data_train = data[data['week_no'] < data['week_no'].max() - validation_weeks]
data_valid = data[data['week_no'] >= data['week_no'].max() - validation_weeks]

In [42]:
users_lvl_1 = data_train.user_id.unique()
users_lvl_2 = data_valid.user_id.unique()
users_lvl_3 = test.user_id.unique()

new_users_lvl_2 = list(set(users_lvl_2) - set(users_lvl_1))
new_users_lvl_3 = list(set(users_lvl_3) - (set(users_lvl_1) | set(users_lvl_2)))

add_to_lvl_2 = list(set(users_lvl_3) - (set(users_lvl_2)))

new_users_lvl_2, new_users_lvl_3, len(add_to_lvl_2)

([], [2325], 75)

In [43]:
n_items_before = data['item_id'].nunique()
data_train = prefilter_items(data_train, item_features=item_features, take_n_popular=5000)
n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 89051 to 5001


In [44]:
user_item_features = get_new_features(data_train)
user_item_features.head(3)

,user_id,item_id,median_sales_hour,median_weekday,mean_visits_interval,mean_check,n_stores,n_items,n_transactions,mean_n_items_basket,max_n_items_basket,std_n_items_basket,mean_n_item_categories_basket,max_n_item_categories_basket,std_n_item_categories_basket
0,1,823721,13.0,4.0,NaN,37.772063,95,177,698,8.47619,21,4.947652,7.355932,18,4.084296
1,1,823990,15.0,6.0,NaN,37.772063,95,177,698,8.47619,21,4.947652,7.355932,18,4.084296
2,1,825123,17.0,3.0,NaN,37.772063,29,177,698,8.47619,21,4.947652,7.355932,18,4.084296


In [45]:
targets_test = get_targets_lvl_2(data_train, data_valid, user_item_features, N, add_to_lvl_2)

print(f'число пользователей: {targets_test.user_id.nunique()}')
print(f'среднее число покупок: {round(targets_test["target"].mean(), 4)}')

targets_test.head(3)

100%|██████████| 5001/5001 [00:00<00:00, 27369.33it/s]


число пользователей: 2272
среднее число покупок: 0.048


,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,manufacturer_freq,...,mean_check,n_stores,n_items,n_transactions,mean_n_items_basket,max_n_items_basket,std_n_items_basket,mean_n_item_categories_basket,max_n_item_categories_basket,std_n_item_categories_basket
0,84,903529,1.0,539,DRUG GM,1,CIGARETTES,CIGARETTES,972976 PK,170.0,...,16.506667,15.0,58.0,163.0,2.928571,14.0,2.788416,2.793103,11.0,2.582625
1,84,903529,1.0,539,DRUG GM,1,CIGARETTES,CIGARETTES,972976 PK,170.0,...,16.506667,15.0,58.0,163.0,2.928571,14.0,2.788416,2.793103,11.0,2.582625
2,84,920025,1.0,764,GROCERY,1,LAUNDRY ADDITIVES,FABRIC SOFTENER LIQUID,60 LOAD,1332.0,...,16.506667,20.0,58.0,163.0,2.928571,14.0,2.788416,2.793103,11.0,2.582625


In [46]:
predictions_lgb_test = model_lgb.predict(targets_test[SELECTED_FEATURES_NAMES].fillna(0))
predictions_cb_test = model_cb.predict_proba(targets_test[SELECTED_FEATURES_NAMES].fillna(0))[:, 1]

In [47]:
preds_test_2_mods = predictions_lgb_test * 0.6 + predictions_cb_test * 0.4
roc_auc_score(targets_test['target'], preds_test_2_mods)

0.9018169156179714

In [48]:
result_test = get_results(test, targets_test, 
                           predictions_lgb_test,
                           predictions_cb_test, 
                           preds_test_2_mods)
result_test.head(3)

,user_id,actual,lgb_recommendations,cb_recommendations,2_mods_recommendations
0,1,"[880007, 883616, 931136, 938004, 940947, 94726...","[1082185, 10457591, 826695, 872137, 1082212]","[872137, 1082185, 940947, 1004906, 9297615]","[1082185, 872137, 1082212, 9297615, 940947]"
1,2,"[820165, 820291, 826784, 826835, 829009, 85784...","[1106523, 916122, 1075368, 9365106, 5569230]","[1106523, 899624, 1068719, 1082185, 1075368]","[1106523, 899624, 1075368, 916122, 1082185]"
2,3,"[827683, 908531, 989069, 1071377, 1080155, 109...","[1082185, 1106523, 866211, 1068719, 979707]","[1082185, 1106523, 866211, 990797, 883003]","[1082185, 1106523, 866211, 990797, 5569230]"


In [49]:
result_test.apply(lambda row: precision_at_k(row['cb_recommendations'], row['actual'], 5), axis=1).mean()

0.23490716180371085

In [50]:
result_test.apply(lambda row: precision_at_k(row['lgb_recommendations'], row['actual'], 5), axis=1).mean()

0.21952254641909552

In [51]:
result_test.apply(lambda row: precision_at_k(row['2_mods_recommendations'], row['actual'], 5), axis=1).mean()

0.23246684350132363

In [53]:
df = result_test[['user_id', '2_mods_recommendations']].copy()
df.rename(columns = {'2_mods_recommendations':'actual'}, inplace = True)
df.to_csv('recommendations.csv', index=False)
df.head(5)

,user_id,actual
0,1,"[1082185, 872137, 1082212, 9297615, 940947]"
1,2,"[1106523, 899624, 1075368, 916122, 1082185]"
2,3,"[1082185, 1106523, 866211, 990797, 5569230]"
3,6,"[1029743, 1082185, 1098844, 1070820, 878996]"
4,7,"[1106523, 1082185, 1029743, 1126899, 1122358]"
